In [1]:
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from tifffile import imread
from skimage.color import rgb2gray
from skimage.transform import rescale
from skimage.exposure import rescale_intensity
from skimage.measure import regionprops
from skimage.measure import regionprops_table
from csbdeep.utils import normalize

from stardist.models import StarDist2D

2024-09-13 10:39:43.359859: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
fpath = "/scratch/indikar_root/indikar1/shared_data/apollo/images/cellsonlyA1.ome.tiff"
img = imread(fpath)
print(f"{img.shape=}")

<tifffile.TiffFrame 388 @157874034> is missing required tags
<tifffile.TiffFile 'cellsonlyA1.ome.tiff'> OME series expected 388 frames, got 389


img.shape=(4, 97, 478, 666, 3)


In [ ]:
def segment_image(img, stardist_model, prob_thresh, nms_thresh):
    """
    Segments cells in an image using StarDist and extracts region properties.

    Args:
        img (numpy.ndarray): The image data with shape (num_channels, time, height, width, 3).
        stardist_model (str): Name of the pretrained StarDist model to use.
        prob_thresh (float): Probability threshold for object detection.
        nms_thresh (float): Non-maximum suppression threshold for object detection.

    Returns:
        tuple: A tuple containing:
            - segments (numpy.ndarray): A labeled image with the same shape as the input image (except for the last dimension), 
                                       where each pixel's value represents the segment it belongs to.
            - region_props (pd.DataFrame): A DataFrame containing the extracted region properties for each detected segment,
                                           including 'Channel' and 'Time' columns.
    """

    region_props = []
    segments = np.zeros((img.shape[:-1]))  # Initialize the segments array

    # Load the StarDist model
    model = StarDist2D.from_pretrained(stardist_model)

    for c in range(img.shape[0]):
        for t in range(img.shape[1]):
            # Extract the image at the current channel and time point, and convert to grayscale
            img_t = img[c, t, :, :, :]
            img_t = rgb2gray(img_t)
            
            # Predict instances (labels and other data) using the StarDist model
            try:
                labels, _ = model.predict_instances(
                    normalize(img_t),
                    prob_thresh=prob_thresh,
                    nms_thresh=nms_thresh
                )
            except:
                continue
                
            print(f"{t=} {len(np.unique(labels))} cells with prob: {prob_thresh}")

            # Store the labels in the segments array
            segments[c, t, :, :] = labels

            # Extract region properties using regionprops_table
            props = regionprops_table(
                labels,
                intensity_image=img_t,
                properties=[
                    'label', 'area', 'bbox_area', 'convex_area', 'eccentricity',
                    'equivalent_diameter', 'extent', 'filled_area', 'major_axis_length',
                    'minor_axis_length', 'orientation', 'perimeter', 'max_intensity',
                    'mean_intensity', 'min_intensity', 'centroid'
                ]
            )

            # Create a DataFrame from the properties, and add 'Channel' and 'Time' columns
            df = pd.DataFrame(props)
            df['Channel'] = c
            df['Time'] = t

            region_props.append(df)

    # Concatenate all the DataFrames into a single DataFrame
    if not region_props:  
        region_props = pd.DataFrame(columns=[
                    'label', 'area', 'bbox_area', 'convex_area', 'eccentricity',
                    'equivalent_diameter', 'extent', 'filled_area', 'major_axis_length',
                    'minor_axis_length', 'orientation', 'perimeter', 'max_intensity',
                    'mean_intensity', 'min_intensity', 'centroid', 'Channel', 'Time',
                ]
            )
    else:
        region_props = pd.concat(region_props)
    
    return segments, region_props
    
    
startdist_model = "2D_versatile_fluo"
prob_thresh = 0.4
nms_thresh = 0.2

segments, region_props = segment_image(
    img, 
    startdist_model,
    prob_thresh,
    nms_thresh
)

region_props.head()

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
t=0 13 cells with prob: 0.4
t=1 11 cells with prob: 0.4
t=2 7 cells with prob: 0.4
t=3 8 cells with prob: 0.4
t=4 8 cells with prob: 0.4
t=5 9 cells with prob: 0.4
t=6 4 cells with prob: 0.4
t=7 7 cells with prob: 0.4
t=8 13 cells with prob: 0.4
t=9 11 cells with prob: 0.4
t=10 11 cells with prob: 0.4
t=11 14 cells with prob: 0.4
t=12 14 cells with prob: 0.4
t=13 18 cells with prob: 0.4
t=14 18 cells with prob: 0.4
t=15 15 cells with prob: 0.4
t=16 27 cells with prob: 0.4
t=17 24 cells with prob: 0.4
t=18 31 cells with prob: 0.4
t=19 36 cells with prob: 0.4
t=20 35 cells with prob: 0.4
t=21 31 cells with prob: 0.4
t=22 36 cells with prob: 0.4
t=23 34 cells with prob: 0.4
t=24 32 cells with prob: 0.4
t=25 40 cells with prob: 0.4
t=26 35 cells with prob: 0.4
t=27 46 cells with prob: 0.4
t=2

In [ ]:
break

In [ ]:
def segment_image(img, stardist_model, prob_thresh, nms_thresh):
    """
    Segments cells in an image using StarDist and extracts region properties.

    Args:
        img (numpy.ndarray): The image data with shape (num_channels, time, height, width, 3).
        stardist_model (str): Name of the pretrained StarDist model to use.
        prob_thresh (float): Probability threshold for object detection.
        nms_thresh (float): Non-maximum suppression threshold for object detection.

    Returns:
        tuple: A tuple containing:
            - segments (numpy.ndarray): A labeled image with the same shape as the input image (except for the last dimension), 
                                       where each pixel's value represents the segment it belongs to.
            - region_props (pd.DataFrame): A DataFrame containing the extracted region properties for each detected segment,
                                           including 'Channel' and 'Time' columns.
    """

    region_props = []
    segments = np.zeros((img.shape[:-1]))  # Initialize the segments array

    # Load the StarDist model
    model = StarDist2D.from_pretrained(stardist_model)

    for c in range(img.shape[0]):
        for t in range(img.shape[1]):
            # Extract the image at the current channel and time point, and convert to grayscale
            img_t = img[c, t, :, :, :]
            img_t = rgb2gray(img_t)

            # Predict instances (labels and other data) using the StarDist model
            try:
                labels, _ = model.predict_instances(
                    normalize(img_t),
                    prob_thresh=prob_thresh,
                    nms_thresh=nms_thresh
                )
            except:
                continue

            # Store the labels in the segments array
            segments[c, t, :, :] = labels

            # Extract region properties using regionprops_table
            props = regionprops_table(
                labels,
                intensity_image=img_t,
                properties=[
                    'label', 'area', 'bbox_area', 'convex_area', 'eccentricity',
                    'equivalent_diameter', 'extent', 'filled_area', 'major_axis_length',
                    'minor_axis_length', 'orientation', 'perimeter', 'max_intensity',
                    'mean_intensity', 'min_intensity', 'centroid'
                ]
            )

            # Create a DataFrame from the properties, and add 'Channel' and 'Time' columns
            df = pd.DataFrame(props)
            df['Channel'] = c
            df['Time'] = t

            region_props.append(df)
            
            break
        break

    # Concatenate all the DataFrames into a single DataFrame
    region_props = pd.concat(region_props)
    
    return segments, region_props



startdist_model = "2D_versatile_fluo"
prob_thresh = 0.4
nms_thresh = 0.2

segment_image(
    img, 
    startdist_model,
    prob_thresh,
    nms_thresh
)